In [ ]:
import numpy as np
import pandas as pd

from generators import (
    SolarGenerator,
    WindGenerator,
    NuclearGenerator,
    CoalGenerator,
    GasGenerator,
)
from utils import get_demand_curve

In [ ]:
get_demand_curve(week_no=7, population=1).plot()

In [ ]:
t = np.arange(0, 7 * 24 + 1, 1)

In [ ]:
gen = SolarGenerator(time_steps=t, installed_capacity=1000, week=6)
df = pd.concat(
    [
        pd.Series(gen.min_power).rename("min"),
        pd.Series(gen.max_power).rename("max"),
    ],
    axis=1,
)
df.plot()
gen.calculate_costs(df.mean(axis=1))

In [ ]:
gen = WindGenerator(time_steps=t, installed_capacity=1000, week=6)
df = pd.concat(
    [
        pd.Series(gen.min_power).rename("min"),
        pd.Series(gen.max_power).rename("max"),
    ],
    axis=1,
)
df.plot()
gen.calculate_costs(df.mean(axis=1))

In [ ]:
gen = NuclearGenerator(time_steps=t, installed_capacity=1000)
df = pd.concat(
    [
        pd.Series(gen.min_power).rename("min"),
        pd.Series(gen.max_power).rename("max"),
    ],
    axis=1,
)
df.plot()
gen.calculate_costs(df.mean(axis=1))

In [ ]:
gen = CoalGenerator(time_steps=t, installed_capacity=1000)
df = pd.concat(
    [
        pd.Series(gen.min_power).rename("min"),
        pd.Series(gen.max_power).rename("max"),
    ],
    axis=1,
)
df.plot()
gen.calculate_costs(df.mean(axis=1))

In [ ]:
gen = GasGenerator(time_steps=t, installed_capacity=1000)
df = pd.concat(
    [
        pd.Series(gen.min_power).rename("min"),
        pd.Series(gen.max_power).rename("max"),
    ],
    axis=1,
)
df.plot()
gen.calculate_costs(df.mean(axis=1))